In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.metrics import precision_score, recall_score, f1_score

# Check and Set Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Load Data
cleaned_data = pd.read_csv("/content/filtered_fault_data.csv")
cleaned_data = cleaned_data.sort_values(by="Createdon")

# Create Lag Features
for lag in range(1, 8):  # Last 7 days
    cleaned_data[f"temp_lag_{lag}"] = cleaned_data["temp"].shift(lag)

# Target Variable (Failure in next 7 days)
cleaned_data["target"] = cleaned_data.groupby("fdr_Id")["fdr_Id"].transform(lambda x: x.shift(-7).notna().astype(int))

# Drop NaN values from shifting
cleaned_data = cleaned_data.dropna()

# Select Features
features = [
    "isemergency", "TAT", "tempmax", "tempmin", "temp", "dew", "humidity", "precip", "precipprob", "precipcover",
    "snow", "snowdepth", "windgust", "windspeed", "winddir", "sealevelpressure", "cloudcover", "visibility",
    "solarradiation", "solarenergy", "uvindex", "severerisk"
]

features += [col for col in cleaned_data.columns if "lag" in col or "rolling" in col]

# Encode Categorical Variables
categorical_features = ["OutageType", "outageSubType", "initialoffreason", "htclosingreason",
                        "fdr_Id", "fdr_name", "grid_name", "Relay", "israintripping"]

label_encoders = {}
for col in categorical_features:
    le = LabelEncoder()
    cleaned_data[col] = le.fit_transform(cleaned_data[col].astype(str))
    label_encoders[col] = le
    features.append(col)

# Normalize Numerical Features
numerical_features = [col for col in features if col not in categorical_features]
scaler = MinMaxScaler()
cleaned_data[numerical_features] = scaler.fit_transform(cleaned_data[numerical_features])

# Convert Data to Sequences
def create_sequences(df, seq_length=4):
    X, y = [], []
    for i in range(len(df) - seq_length - 1):
        X.append(df[features].iloc[i:i+seq_length].values)
        y.append(df["target"].iloc[i+seq_length])
    return torch.tensor(X, dtype=torch.float32), torch.tensor(y, dtype=torch.float32)

X, y = create_sequences(cleaned_data)

# Move Data to GPU
X, y = X.to(device), y.to(device)

# Train-Test Split
split_idx = int(len(X) * 0.8)
X_train, X_test = X[:split_idx], X[split_idx:]
y_train, y_test = y[:split_idx], y[split_idx:]

# DataLoader
batch_size = 64
train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Transformer Model
class TransformerModel(nn.Module):
    def __init__(self, input_dim, num_heads=4, num_layers=3, hidden_dim=128):
        super(TransformerModel, self).__init__()
        self.embedding = nn.Linear(input_dim, hidden_dim)  # Project input to hidden_dim
        self.positional_encoding = nn.Parameter(torch.zeros(1, 50, hidden_dim))  # Learnable positional encoding
        encoder_layer = nn.TransformerEncoderLayer(d_model=hidden_dim, nhead=num_heads, dim_feedforward=256)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        self.fc = nn.Linear(hidden_dim, 1)  # Output layer for binary classification

    def forward(self, x):
        x = self.embedding(x) + self.positional_encoding[:, :x.size(1), :]
        x = self.transformer_encoder(x)
        x = x[:, -1, :]  # Use last time step's output
        return torch.sigmoid(self.fc(x))

# Instantiate Model
model = TransformerModel(input_dim=X.shape[2]).to(device)
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train Model
for epoch in range(4):
    model.train()
    total_loss = 0
    for batch_X, batch_y in train_loader:
        batch_X, batch_y = batch_X.to(device), batch_y.to(device)
        optimizer.zero_grad()

        # Forward pass
        output = model(batch_X).squeeze()
        loss = criterion(output, batch_y)
        total_loss += loss.item()

        # Backward pass
        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch+1}, Loss: {total_loss / len(train_loader):.4f}")

# Evaluate Model
model.eval()
y_pred_list, y_test_list = [], []

with torch.no_grad():
    for batch_X, batch_y in test_loader:
        batch_X, batch_y = batch_X.to(device), batch_y.to(device)
        output = model(batch_X).squeeze()
        y_pred_list.extend(output.cpu().numpy())
        y_test_list.extend(batch_y.cpu().numpy())

# Convert Predictions to Binary
y_pred_binary = (np.array(y_pred_list) > 0.5).astype(int)
y_test_cpu = np.array(y_test_list)

# Compute Metrics
precision = precision_score(y_test_cpu, y_pred_binary)
recall = recall_score(y_test_cpu, y_pred_binary)
f1 = f1_score(y_test_cpu, y_pred_binary)

print(f"Precision: {precision:.4f}, Recall: {recall:.4f}, F1 Score: {f1:.4f}")


Using device: cuda


<ipython-input-1-353206a3a352>:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_data[col] = le.fit_transform(cleaned_data[col].astype(str))
<ipython-input-1-353206a3a352>:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_data[col] = le.fit_transform(cleaned_data[col].astype(str))
<ipython-input-1-353206a3a352>:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

In [ ]:
import pandas as pd
import torch

def predict_feeder_failures(model, data, month, features, device="cuda"):
    """
    Predicts which feeders are likely to fail in a given month.

    Parameters:
        model (torch.nn.Module): Trained Transformer model.
        data (pd.DataFrame): Cleaned and preprocessed dataset.
        month (str): Target month in 'YYYY-MM' format.
        features (list): List of feature column names used in training.
        device (str): 'cuda' or 'cpu' depending on availability.

    Returns:
        List of feeder IDs predicted to fail in the given month.
    """
    # Convert 'Createdon' to datetime format and filter data for the given month
    data["Createdon"] = pd.to_datetime(data["Createdon"])
    month_data = data[data["Createdon"].dt.strftime("%Y-%m") == month]

    if month_data.empty:
        print(f"No data available for {month}.")
        return []

    # Ensure feature columns exist
    month_data = month_data.dropna(subset=features)

    # Convert data into sequences (assuming sequence length = 4)
    def create_sequences(df, seq_length=4):
        X, fdr_ids = [], []
        for i in range(len(df) - seq_length):
            X.append(df[features].iloc[i:i+seq_length].values)
            fdr_ids.append(df["fdr_Id"].iloc[i + seq_length])  # Capture Feeder ID
        return torch.tensor(X, dtype=torch.float32), fdr_ids

    X, fdr_ids = create_sequences(month_data)

    if len(X) == 0:
        print(f"Not enough sequential data for predictions in {month}.")
        return []

    # Move data to device
    X = X.to(device)

    # Predict using the model
    model.eval()
    with torch.no_grad():
        predictions = model(X).squeeze().cpu().numpy()

    # Convert predictions to binary (threshold > 0.5 means failure)
    predicted_failures = (predictions > 0.5).astype(int)

    # Get feeder IDs of predicted failures
    failed_feeders = [fdr for fdr, pred in zip(fdr_ids, predicted_failures) if pred == 1]

    return list(set(failed_feeders))  # Remove duplicates


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/valida

Feeders predicted to fail in February 2025: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190]


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/valida

In [ ]:
month = "2025-03"  # Example month
failed_feeders = predict_feeder_failures(model, cleaned_data, month, features)
print(f"Feeders predicted to fail in {month}: {failed_feeders}")